In [1]:
import numpy as np
import cvxpy as cp 
import matplotlib.pyplot as plt
from scipy.special import logsumexp, softmax
%matplotlib inline
%config InlineBackend.figure_format='retina'
from IPython.display import clear_output
import time
clear_output()

In [2]:
"""
data 
"""

np.random.seed(23824)

TOLDIV = 5e-10
MIT = 2.5e3

d = int(1e3)
fade_cand = -1
fade = max(-10,min(1,fade_cand))
p = 1
R_def = 10
temp = 1e-2

N_LSE = int(5e4)
LV,UV = -5,5

normalization = False

In [3]:
# from INRIA 
def proj_simplex_(vec, R=1,tol=1e-3):
    """ 
    References
    ----------
    [1] Efficient Projections onto the .1-Ball for Learning in High Dimensions
        John Duchi, Shai Shalev-Shwartz, Yoram Singer, and Tushar Chandra.
        International Conference on Machine Learning (ICML 2008)
        http://www.cs.berkeley.edu/~jduchi/projects/DuchiSiShCh08.pdf
    """
    assert R > 0, "radius R must be strictly positive (%d <= 0)" % R
    d, = vec.shape  # will raise ValueError if vec is not 1-D
    if abs(vec.sum() - R)<=tol and np.alltrue(vec >= 0):
        return vec
    sorted_vec = np.sort(vec)[::-1]
    cssv = np.cumsum(sorted_vec)
    buf = np.nonzero(sorted_vec * np.arange(1, d+1) > (cssv - R))[0][-1]
    shift = float(cssv[buf] - R) / (1+buf)
    w = (vec - shift).clip(min=0)
    return w

def proj_ball_(x,xi_embd=np.zeros(d),nm_power=p,R=R_def,tol=5e-4):
    dxxi_embd = x-xi_embd
    abs_dxxi_embd = np.abs(dxxi_embd)
    if nm_power==1:
        if np.sum(abs_dxxi_embd)<=R+tol:
            return x
        return xi_embd + proj_simplex_(abs_dxxi_embd,R)*np.sign(dxxi_embd)
    elif nm_power==np.inf:
        if max(abs_dxxi_embd)<=R+tol:
            return x
        return xi_embd + np.minimum(R,np.maximum(-R,dxxi_embd))
    elif nm_power==2:
        buf = (np.sum(abs_dxxi_embd**2))**(1/2)
        if buf<=R+tol:
            return x
        return xi_embd + dxxi_embd*R/max(R,buf)
    else:
        return None
    
k_inner_max_def = int(5e4)
def prox_bundle_update(G,ftilde,z,L,verb=False,k_inner_max=k_inner_max_def):
    local_bs = len(G)
    if local_bs==0:
        return proj_ball_(z),None
    elif local_bs==1:
        return proj_ball_(z-G[0]/L),np.array([1])
    else:
        weights = np.zeros(local_bs)
        weights[-1] = 1
        slack = 1
        k_inner = 0
        u_ = proj_ball_(z-weights@G/L)
        slack = max(G@u_+ftilde)-((weights@G)@u_ + weights@ftilde)
        while slack>1e-4 and k_inner<k_inner_max:
            grad_ = ftilde+G@u_
            changeID = np.argmax(grad_)
            weights =  k_inner/(k_inner+2)*weights
            weights[changeID]+=2/(k_inner+2)
            u_ = proj_ball_(z-weights@G/L)
            slack = max(G@u_+ftilde)-((weights@G)@u_ + weights@ftilde)
            k_inner += 1
        if slack>1e-4:
            print('inexact prox | slack = '+str(slack))
        return u_,weights

"""

active components

"""

def selection(x_new,G1,f1tilde,tol_act=5e-5):
    vals = np.array(G1)@x_new+np.array(f1tilde)
    buf = np.where(vals>=np.max(vals)-tol_act)[0]
    if len(buf)>0:
        G1_buf,f1tilde_buf = [],[]
        for elem in buf:
            G1_buf.append(G1[int(elem)])
            f1tilde_buf.append(f1tilde[int(elem)])
        return G1_buf,f1tilde_buf
    else:
        return [],[]

In [4]:
# from deepseek

def split_matrix(U):
    # Compute the eigenvalues and eigenvectors of U
    eigvals, eigvecs = np.linalg.eigh(U)
    
    # Split the eigenvalues into positive and negative parts
    lambda_plus = np.maximum(eigvals, 0)
    lambda_minus = np.maximum(-eigvals, 0)
    
    # Construct the diagonal matrices for positive and negative parts
    Lambda_plus = np.diag(lambda_plus)
    Lambda_minus = np.diag(lambda_minus)
    
    # Reconstruct U1 and U2 using the eigenvectors
    U1 = eigvecs @ Lambda_plus @ eigvecs.T
    U2 = eigvecs @ Lambda_minus @ eigvecs.T
    
    return (U1+U1.T)/2, (U2+U2.T)/2

In [5]:
def SDCe(MO,f1,f2,Psi,F,x_init,L_init,desc_prop=.1,tau_loc=2/3,ns_trig=5,dc_steps=int(200),\
            reset_after_ls=False,max_iter=1000,verb=False,vb_freq=50):
    
    # inits.
    tau_ext = .001
    x_stability_prev_next,x_stability_prev,x_stability,x,L = x_init.copy(),x_init.copy(),x_init.copy(),x_init.copy(),L_init
    k = 0
    div_succ_it = np.inf
    num_streak_success = 0
    L_min,L_max,L_prev = 1,1e16,0
    th,th_prev,varsigma = 1,0,0
    
    DCA_useful_iters = 0

    cond = True
    
    while k<max_iter and div_succ_it>TOLDIV and L<L_max:
                
        # IO
        y = x_stability+min(np.sqrt(L_prev*(1-tau_ext)/L),varsigma)*(x_stability_prev_next-x_stability_prev)
        f1y,g1y = f1(y,mode='full')
        f1y_tilde = f1y-np.sum(g1y*y)
        if cond:
            f2x,g2x = f2(x=x_stability,mode='full')
            f2x_stability,g2x_stability = f2x,g2x
            f2x_stability_tilde = f2x_stability-np.sum(g2x_stability*x_stability)
            val = f1(x_stability,'value')-f2x_stability+Psi(x_stability)

        cond = False
        
        if num_streak_success>=ns_trig:
            L = max(L_min+0.01,L*tau_loc)
            if reset_after_ls:
                num_streak_success = 0
            
        # MO 
        x_stability_plus,x,DCA_inner = MO(y,g1y,f1y_tilde,g2x_stability,f2x_stability_tilde,L,max_steps=dc_steps,min_decr_inner=5e-4)

        # IO 
        f1x,g1x = f1(x,mode='full')
        val_new = f1x-f2(x,'value')+Psi(x)
        
        # potential sufficient decrease check 
        div_ = np.sum((x-y)**2)/2
        div_x_stab_x_stab_plus = np.sum((x_stability_plus-x_stability)**2)/2
        cond = val_new + L*div_x_stab_x_stab_plus <= val + L*np.sum((y-x_stability)**2)/2 - L_min*desc_prop*div_

        if cond==False:
            L = min(L_max,L/tau_loc)
            num_streak_success = 0 # reset counter
        else:
            num_streak_success += 1
                
        # updates
        if cond:
            k += 1
            L_prev = L
            DCA_useful_iters += DCA_inner>0
            div_succ_it = np.sum((x-x_stability)**2)/2
            x_stability_prev = x_stability
            x_stability_prev_next = x_stability_plus
            x_stability = x.copy()
            val = val_new
            new_th = (1+np.sqrt(1+4*th**2))/2
            th_prev = th
            th = new_th
            varsigma = (th_prev-1)/th

        
        if verb and k%vb_freq==0 and cond:
            print('#'+str(k)+' itrs. | Fval = '+'{:.4e}'.format(val_new)+' ; div. '+'{:.2e}'.format(div_succ_it)+' ; L = '+'{:.3e}'.format(L)+' ; #DC iters = '+str(DCA_useful_iters))
    print(' ')
    print('HALT #'+str(k)+' iters. | Fval = '+str(val_new)+' ; div. '+str(div_succ_it)+' ; #DC iters = '+str(DCA_useful_iters)) 
            
    return x,val,k,DCA_useful_iters

In [6]:
def ASDCe(BMO,f1,f2,Psi,F,x_init,L_init,desc_prop=.1,tau_loc=2/3,ns_trig=5,dc_steps=int(200),\
            reset_after_ls=False,filter_out=True,bs_max=max(min(d+5,1000),100),max_iter=1000,verb=False,vb_freq=50):
    
    # inits.
    tau_ext = .001
    x_stability_prev_next,x_stability_prev,x_stability,x,L = x_init.copy(),x_init.copy(),x_init.copy(),x_init.copy(),L_init
    k = 0
    div_succ_it = np.inf
    num_streak_success = 0
    L_min,L_max,L_prev = 1,1e16,0
    th,th_prev,varsigma = 1,0,0
    
    DCA_useful_iters = 0
    
    G1 = []
    f1tilde = []

    cond = True
    
    while k<max_iter and div_succ_it>TOLDIV and L<L_max:
                
        # IO
        y = x_stability+min(np.sqrt(L_prev*(1-tau_ext)/L),varsigma)*(x_stability_prev_next-x_stability_prev)
        f1y,g1y = f1(y,mode='full')
        f1y_tilde = f1y-np.sum(g1y*y)
        if cond:
            f2x,g2x = f2(x=x_stability,mode='full')
            f2x_stability,g2x_stability = f2x,g2x
            f2x_stability_tilde = f2x_stability-np.sum(g2x_stability*x_stability)
            val = f1(x_stability,'value')-f2x_stability+Psi(x_stability)
            
        G1 = G1[-(bs_max-1):]
        G1.append(g1y)
        f1tilde = f1tilde[-(bs_max-1):]
        f1tilde.append(f1y_tilde)

        cond = False
        
        if num_streak_success>=ns_trig:
            L = max(L_min+0.01,L*tau_loc)
            if reset_after_ls:
                num_streak_success = 0
            
        # MO 
        B_size = len(G1)
        x_stability_plus,weights,x,DCA_inner = BMO(y,G1,f1tilde,g2x_stability,f2x_stability_tilde,L,max_steps=dc_steps,min_decr_inner=5e-4)

        # IO 
        f1x,g1x = f1(x,mode='full')
        val_new = f1x-f2(x,'value')+Psi(x)
        
        # potential sufficient decrease check 
        div_ = np.sum((x-y)**2)/2
        div_x_stab_x_stab_plus = np.sum((x_stability_plus-x_stability)**2)/2
        cond = val_new + L*div_x_stab_x_stab_plus <= val + L*np.sum((y-x_stability)**2)/2 - L_min*desc_prop*div_

        if cond==False:
            L = min(L_max,L/tau_loc)
            num_streak_success = 0 # reset counter
        else:
            num_streak_success += 1
                
        # updates
        if cond:
            k += 1
            L_prev = L
            DCA_useful_iters += DCA_inner>0
            div_succ_it = np.sum((x-x_stability)**2)/2
            x_stability_prev = x_stability
            x_stability_prev_next = x_stability_plus
            x_stability = x.copy()
            val = val_new
            new_th = (1+np.sqrt(1+4*th**2))/2
            th_prev = th
            th = new_th
            varsigma = (th_prev-1)/th
            
            # bundle management
            if filter_out:
                G1,f1tilde = selection(x_stability,np.array(G1),f1tilde)
                
        else:
            
            # bundle management
            aggregate_g = weights@G1
            aggregate_ftilde = weights@f1tilde
            ## getting rid of stability center info that will be reintroduced hereafter ##
            G1 = G1[:-1]
            f1tilde = f1tilde[:-1] 
            G1.append(aggregate_g)
            f1tilde.append(aggregate_ftilde)
            G1.append(g1x)
            f1tilde.append(f1x-np.sum(g1x*x))

        
        if verb and k%vb_freq==0 and cond:
            print('#'+str(k)+' itrs. | Fval = '+'{:.4e}'.format(val_new)+' ; div. '+'{:.2e}'.format(div_succ_it)+' ; L = '+'{:.3e}'.format(L)+' ; #DC iters = '+str(DCA_useful_iters)+' ; B_size = '+str(B_size))
    print(' ')
    print('HALT #'+str(k)+' iters. | Fval = '+str(val_new)+' ; div. '+str(div_succ_it)+' ; #DC iters = '+str(DCA_useful_iters)) 
            
    return x,val,k,DCA_useful_iters

In [7]:
def SDC(MO,f1,f2,Psi,F,x_init,L_init,desc_prop=.1,tau_loc=2/3,ns_trig=5,dc_steps=int(200),\
            reset_after_ls=False,max_iter=1000,verb=False,vb_freq=50):
    
    # inits.
    x_stability,x,L = x_init.copy(),x_init.copy(),L_init
    k = 0
    div_succ_it = np.inf
    num_streak_success = 0
    L_min,L_max,L_prev = 1,1e16,0
    
    DCA_useful_iters = 0

    cond = True
    
    while k<max_iter and div_succ_it>TOLDIV and L<L_max:
                
        # IO
        if cond:
            y = x_stability.copy()
            f1y,g1y = f1(y,mode='full')
            f1y_tilde = f1y-np.sum(g1y*y)
            f2x,g2x = f2(x=x_stability,mode='full')
            f2x_stability,g2x_stability = f2x,g2x
            f2x_stability_tilde = f2x_stability-np.sum(g2x_stability*x_stability)
            val = f1(x_stability,'value')-f2x_stability+Psi(x_stability)

        cond = False
        
        if num_streak_success>=ns_trig:
            L = max(L_min+0.01,L*tau_loc)
            if reset_after_ls:
                num_streak_success = 0
            
        # MO 
        x_stability_plus,x,DCA_inner = MO(y,g1y,f1y_tilde,g2x_stability,f2x_stability_tilde,L,max_steps=dc_steps,min_decr_inner=5e-4)

        # IO 
        f1x,g1x = f1(x,mode='full')
        val_new = f1x-f2(x,'value')+Psi(x)
        
        # potential sufficient decrease check 
        div_ = np.sum((x-y)**2)/2
        div_x_stab_x_stab_plus = np.sum((x_stability_plus-x_stability)**2)/2
        cond = val_new + L*div_x_stab_x_stab_plus <= val - L_min*desc_prop*div_

        if cond==False:
            L = min(L_max,L/tau_loc)
            num_streak_success = 0 # reset counter
        else:
            num_streak_success += 1
                
        # updates
        if cond:
            k += 1
            L_prev = L
            DCA_useful_iters += DCA_inner>0
            div_succ_it = np.sum((x-x_stability)**2)/2
            x_stability = x.copy()
            val = val_new
        
        if verb and k%vb_freq==0 and cond:
            print('#'+str(k)+' itrs. | Fval = '+'{:.4e}'.format(val_new)+' ; div. '+'{:.2e}'.format(div_succ_it)+' ; L = '+'{:.3e}'.format(L)+' ; #DC iters = '+str(DCA_useful_iters))
    print(' ')
    print('HALT #'+str(k)+' iters. | Fval = '+str(val_new)+' ; div. '+str(div_succ_it)+' ; #DC iters = '+str(DCA_useful_iters)) 
            
    return x,val,k,DCA_useful_iters

In [8]:
def ASDC(BMO,f1,f2,Psi,F,x_init,L_init,desc_prop=.1,tau_loc=2/3,ns_trig=5,dc_steps=int(200),\
            reset_after_ls=False,filter_out=True,bs_max=max(min(d+5,1000),100),max_iter=1000,verb=False,vb_freq=50):
    
    # inits.
    x_stability,x,L = x_init.copy(),x_init.copy(),L_init
    k = 0
    div_succ_it = np.inf
    num_streak_success = 0
    L_min,L_max,L_prev = 1,1e16,0
    
    DCA_useful_iters = 0
    
    G1 = []
    f1tilde = []

    cond = True
    
    while k<max_iter and div_succ_it>TOLDIV and L<L_max:
                
        # IO
        if cond:
            y = x_stability.copy()
            f1y,g1y = f1(y,mode='full')
            f1y_tilde = f1y-np.sum(g1y*y)
            f2x,g2x = f2(x=x_stability,mode='full')
            f2x_stability,g2x_stability = f2x,g2x
            f2x_stability_tilde = f2x_stability-np.sum(g2x_stability*x_stability)
            val = f1(x_stability,'value')-f2x_stability+Psi(x_stability)
            
        G1 = G1[-(bs_max-1):]
        G1.append(g1y)
        f1tilde = f1tilde[-(bs_max-1):]
        f1tilde.append(f1y_tilde)

        cond = False
        
        if num_streak_success>=ns_trig:
            L = max(L_min+0.01,L*tau_loc)
            if reset_after_ls:
                num_streak_success = 0
            
        # MO 
        x_stability_plus,weights,x,DCA_inner = BMO(y,G1,f1tilde,g2x_stability,f2x_stability_tilde,L,max_steps=dc_steps,min_decr_inner=5e-4)

        # IO 
        f1x,g1x = f1(x,mode='full')
        val_new = f1x-f2(x,'value')+Psi(x)
        
        # potential sufficient decrease check 
        div_ = np.sum((x-y)**2)/2
        div_x_stab_x_stab_plus = np.sum((x_stability_plus-x_stability)**2)/2
        cond = val_new + L*div_x_stab_x_stab_plus <= val - L_min*desc_prop*div_

        if cond==False:
            L = min(L_max,L/tau_loc)
            num_streak_success = 0 # reset counter
        else:
            num_streak_success += 1
                
        # updates
        if cond:
            k += 1
            L_prev = L
            DCA_useful_iters += DCA_inner>0
            div_succ_it = np.sum((x-x_stability)**2)/2
            x_stability = x.copy()
            val = val_new
            
            # bundle management
            if filter_out:
                G1,f1tilde = selection(x_stability,np.array(G1),f1tilde)
                
        else:
            
            # bundle management
            aggregate_g = weights@G1
            aggregate_ftilde = weights@f1tilde
            ## getting rid of stability center info that will be reintroduced hereafter ##
            G1 = G1[:-1]
            f1tilde = f1tilde[:-1] 
            G1.append(aggregate_g)
            f1tilde.append(aggregate_ftilde)
            G1.append(g1x)
            f1tilde.append(f1x-np.sum(g1x*x))
        
        if verb and k%vb_freq==0 and cond:
            print('#'+str(k)+' itrs. | Fval = '+'{:.4e}'.format(val_new)+' ; div. '+'{:.2e}'.format(div_succ_it)+' ; L = '+'{:.3e}'.format(L)+' ; #DC iters = '+str(DCA_useful_iters))
    print(' ')
    print('HALT #'+str(k)+' iters. | Fval = '+str(val_new)+' ; div. '+str(div_succ_it)+' ; #DC iters = '+str(DCA_useful_iters)) 
            
    return x,val,k,DCA_useful_iters

In [9]:
import pandas as pd

In [10]:
NPROBS = int(5)
REPS = int(5)
TL = 2/3

times = []
values = []
repetitions = []
extraps = []
method = []
num_iters = []
DC_updates = []
ls = []
probIDs = []
dca_useful_iters = []

for probID in range(NPROBS):
    

    if normalization:
        betas = np.random.randint(LV,UV+1,(N_LSE,d))/max(UV,LV)
    else:
        betas = np.random.randint(LV,UV+1,(N_LSE,d))

    U = np.random.uniform(-2,2,(d,d))

    _, U2 = split_matrix(U)
    U2 = (1-fade)*U2
    U2 = (U2+U2.T)/2
    L_ref = 1000.0
    print(' ')
    print('PID #'+str(probID))
    print(' ')

    """
    information oracles
    """


    def f1_oracle(x,mode='value'):
        buf = (betas@x)/temp
        if mode=='value':
            return temp*logsumexp(buf)
        elif mode=='gradient':
            return softmax(buf)@betas
        else:
            return temp*logsumexp(buf),softmax(buf)@betas

    def f2_oracle(x,mode='value'):
        U2x = U2@x
        if mode=='value':
            return (1/2)*np.sum(U2x*x)
        elif mode=='gradient':
            return U2x
        else:
            return (1/2)*np.sum(U2x*x),U2x

    def Psi_oracle(x):
        if np.linalg.norm(x,p)<=R_def+3e-3:
            return 0
        else:
            return np.inf

    def F_oracle(x):
        return f1_oracle(x)-f2_oracle(x)+Psi_oracle(x)
    
    """
    oracle 

    approx solves: (subProblem) min mod(u) := f1(y) + <grad f1(y),u-y> + L_hat/2*||u-y||_2^2 + Psi(u) - f2(u)

    ensures with (x)_+ = arg min f1(y) + <grad f1(y),u-y> + L_hat/2*||u-y||_2^2 + Psi(u) - f2(x)-<g2(x),u-x>
    that

    output x_+ is a (approx) critical point of (subProblem) (i)
    output x_+ such that mod(x_+) <= mod((x)_+) (ii)


    """

    def MO_oracle(y,g1y,f1y_tilde,g2x,f2x_tilde,L_hat,max_steps=0,min_decr_inner=5e-4):
        mod = lambda u: f1y_tilde+np.sum(g1y*y)-f2_oracle(u,'value')+Psi_oracle(u)+L_hat*np.sum((u-y)**2)/2
        x_plus = proj_ball_(y-(g1y-g2x)/L_hat)
        x_ = x_plus.copy()
        modval = mod(x_)
        k_inner,cond = 0,True
        while k_inner<max_steps and cond:
            g2x_ = f2_oracle(x_,'gradient')
            x_next = proj_ball_(y-(g1y-g2x_)/L_hat)
            modval_next = mod(x_next)
            decr = modval-modval_next
            if decr>min_decr_inner:
                k_inner += 1
                x_ = x_next.copy()
                modval = modval_next
            else:
                cond = False
                if modval_next<modval:
                    x_ = x_next.copy()
        return x_plus,x_,k_inner
    
    
    def BMO_oracle(y,G1,f1tilde,g2x,f2x_tilde,L_hat,max_steps=0,min_decr_inner=5e-4):
        mod = lambda u: np.max(f1tilde+G1@y)-f2_oracle(u,'value')+Psi_oracle(u)+L_hat*np.sum((u-y)**2)/2
        x_plus,weights = prox_bundle_update(np.array(G1)-np.outer(np.ones(len(G1)),g2x),np.array(f1tilde)-f2x_tilde,z=y,L=L_hat)
        x_ = x_plus.copy()
        modval = mod(x_)
        k_inner,cond = 0,True
        while k_inner<max_steps and cond:
            f2x_,g2x_ = f2_oracle(x_,'full')
            x_next,weights_next = prox_bundle_update(np.array(G1)-np.outer(np.ones(len(G1)),g2x_),np.array(f1tilde)-(f2x_-np.sum(g2x_*x_)),z=y,L=L_hat)
            modval_next = mod(x_next)
            decr = modval-modval_next
            if decr>min_decr_inner:
                k_inner += 1
                x_ = x_next.copy()
                weights = weights_next.copy()
                modval = modval_next
            else:
                cond = False
                if modval_next<modval:
                    x_ = x_next.copy()
                    weights = weights_next.copy()
        return x_plus,weights,x_,k_inner

    for rep in range(REPS):

        x_random = proj_ball_(np.random.uniform(-3,3,d))

        print('repID: '+str(rep))
        print(' ')
        
        
        print(' ')

        print('go ASDCe')
        time_start_ASDCe = time.time()
        x_out_ASDCe,F_out_ASDCe,k_out_ASDCe,dca_used_ASDCe = ASDCe(BMO=BMO_oracle,f1=f1_oracle,f2=f2_oracle,Psi=Psi_oracle,F=F_oracle,x_init=x_random,L_init=L_ref,dc_steps=int(200),desc_prop=.1,tau_loc=TL,ns_trig=3,\
                reset_after_ls=False,max_iter=MIT,verb=True,vb_freq=1)

        time_end_ASDCe = time.time()

        repetitions.append(rep)
        times.append(time_end_ASDCe-time_start_ASDCe)
        values.append(F_out_ASDCe)
        method.append('ASDCe')
        num_iters.append(k_out_ASDCe)
        extraps.append('yes')
        DC_updates.append('yes')
        ls.append('yes')
        probIDs.append(probID)
        dca_useful_iters.append(dca_used_ASDCe)
        

        print(' ')

        print('go SDCe-LS')
        time_start_SDCe_LS = time.time()
        x_out_SDCe_LS,F_out_SDCe_LS,k_out_SDCe_LS,dca_used_SDCe_LS = SDCe(MO=MO_oracle,f1=f1_oracle,f2=f2_oracle,Psi=Psi_oracle,F=F_oracle,x_init=x_random,L_init=L_ref,dc_steps=int(200),desc_prop=.1,tau_loc=TL,ns_trig=3,\
                reset_after_ls=False,max_iter=MIT,verb=True,vb_freq=10)

        time_end_SDCe_LS = time.time()

        repetitions.append(rep)
        times.append(time_end_SDCe_LS-time_start_SDCe_LS)
        values.append(F_out_SDCe_LS)
        method.append('SDCe-LS')
        num_iters.append(k_out_SDCe_LS)
        extraps.append('yes')
        DC_updates.append('yes')
        ls.append('yes')
        probIDs.append(probID)
        dca_useful_iters.append(dca_used_SDCe_LS)


        print(' ')

        print('go SCe-LS')
        time_start_SCe_LS = time.time()
        x_out_SCe_LS,F_out_SCe_LS,k_out_SCe_LS,dca_used_SCe_LS = SDCe(MO=MO_oracle,f1=f1_oracle,f2=f2_oracle,Psi=Psi_oracle,F=F_oracle,x_init=x_random,L_init=L_ref,dc_steps=int(0),desc_prop=.1,tau_loc=TL,ns_trig=3,\
                reset_after_ls=False,max_iter=MIT,verb=True,vb_freq=10)

        time_end_SCe_LS = time.time()

        repetitions.append(rep)
        times.append(time_end_SCe_LS-time_start_SCe_LS)
        values.append(F_out_SCe_LS)
        method.append('SCe-LS')
        num_iters.append(k_out_SCe_LS)
        extraps.append('yes')
        DC_updates.append('no')
        ls.append('yes')
        probIDs.append(probID)
        dca_useful_iters.append(dca_used_SCe_LS)
        
        
        print(' ')

        print('go ASDC')
        time_start_ASDC = time.time()
        x_out_ASDC,F_out_ASDC,k_out_ASDC,dca_used_ASDC = ASDC(BMO=BMO_oracle,f1=f1_oracle,f2=f2_oracle,Psi=Psi_oracle,F=F_oracle,x_init=x_random,L_init=L_ref,dc_steps=int(200),desc_prop=.1,tau_loc=TL,ns_trig=3,\
                reset_after_ls=False,max_iter=MIT,verb=True,vb_freq=10)

        time_end_ASDC = time.time()

        repetitions.append(rep)
        times.append(time_end_ASDC-time_start_ASDC)
        values.append(F_out_ASDC)
        method.append('ASDC')
        num_iters.append(k_out_ASDC)
        extraps.append('yes')
        DC_updates.append('yes')
        ls.append('yes')
        probIDs.append(probID)
        dca_useful_iters.append(dca_used_ASDC)


        print(' ')


        print('go SDC-LS')
        time_start_SDC_LS = time.time()
        x_out_SDC_LS,F_out_SDC_LS,k_out_SDC_LS,dca_used_SDC_LS = SDC(MO=MO_oracle,f1=f1_oracle,f2=f2_oracle,Psi=Psi_oracle,F=F_oracle,x_init=x_random,L_init=L_ref,dc_steps=int(200),desc_prop=.1,tau_loc=TL,ns_trig=3,\
                reset_after_ls=False,max_iter=MIT,verb=True,vb_freq=10)

        time_end_SDC_LS = time.time()

        repetitions.append(rep)
        times.append(time_end_SDC_LS-time_start_SDC_LS)
        values.append(F_out_SDC_LS)
        method.append('SDC-LS')
        num_iters.append(k_out_SDC_LS)
        extraps.append('no')
        DC_updates.append('yes')
        ls.append('yes')
        probIDs.append(probID)
        dca_useful_iters.append(dca_used_SDC_LS)

        print( '')

        print('go SC-LS')
        time_start_SC_LS = time.time()
        x_out_SC_LS,F_out_SC_LS,k_out_SC_LS,dca_used_SC_LS = SDC(MO=MO_oracle,f1=f1_oracle,f2=f2_oracle,Psi=Psi_oracle,F=F_oracle,x_init=x_random,L_init=L_ref,dc_steps=int(0),desc_prop=.1,tau_loc=TL,ns_trig=3,\
                reset_after_ls=False,max_iter=MIT,verb=True,vb_freq=10)

        time_end_SC_LS = time.time()

        repetitions.append(rep)
        times.append(time_end_SC_LS-time_start_SC_LS)
        values.append(F_out_SC_LS)
        method.append('SC-LS')
        num_iters.append(k_out_SC_LS)
        extraps.append('no')
        DC_updates.append('no')
        ls.append('yes')
        probIDs.append(probID)
        dca_useful_iters.append(dca_used_SC_LS)

        print(' ')




        subdf = pd.DataFrame(data={'repID':repetitions,'vals':values,'times':times,'method':method,'iters':num_iters,\
                                   'extrapolation':extraps,'dc_updates':DC_updates,'linesarch':ls,'pID':probIDs,'num_used_dc_steps':dca_useful_iters})    

        if fade<0:
            subdf.to_csv('benchmark_methods_concave_LSE_QP_d_'+str(d)+'.csv')
        elif fade>0:
            subdf.to_csv('benchmark_methods_convex_LSE_QP_d_'+str(d)+'.csv')
        else:
            subdf.to_csv('benchmark_methods_neutral_LSE_QP_d_'+str(d)+'.csv')
            
    print(' ')

 
PID #0
 
repID: 0
 
 
go ASDCe
#1 itrs. | Fval = -8.9518e+00 ; div. 9.67e-04 ; L = 1.000e+03 ; #DC iters = 0 ; B_size = 1
#2 itrs. | Fval = -1.0026e+01 ; div. 6.08e-04 ; L = 1.000e+03 ; #DC iters = 0 ; B_size = 2
#3 itrs. | Fval = -1.0748e+01 ; div. 3.57e-04 ; L = 1.500e+03 ; #DC iters = 0 ; B_size = 4
#4 itrs. | Fval = -1.1350e+01 ; div. 4.12e-04 ; L = 1.500e+03 ; #DC iters = 0 ; B_size = 3
#5 itrs. | Fval = -1.1942e+01 ; div. 2.38e-04 ; L = 3.375e+03 ; #DC iters = 0 ; B_size = 6
#6 itrs. | Fval = -1.2488e+01 ; div. 2.20e-04 ; L = 3.375e+03 ; #DC iters = 0 ; B_size = 3
#7 itrs. | Fval = -1.2933e+01 ; div. 1.70e-04 ; L = 5.062e+03 ; #DC iters = 0 ; B_size = 5
#8 itrs. | Fval = -1.3385e+01 ; div. 1.31e-04 ; L = 7.594e+03 ; #DC iters = 0 ; B_size = 4
#9 itrs. | Fval = -1.3802e+01 ; div. 1.16e-04 ; L = 7.594e+03 ; #DC iters = 0 ; B_size = 2
#10 itrs. | Fval = -1.4145e+01 ; div. 9.01e-05 ; L = 1.139e+04 ; #DC iters = 0 ; B_size = 4
#11 itrs. | Fval = -1.4484e+01 ; div. 7.86e-05 ; L = 1.1

 
HALT #302 iters. | Fval = -1627.9450724198048 ; div. 0.0 ; #DC iters = 6

go SC-LS
#10 itrs. | Fval = -1.0075e+01 ; div. 8.95e-08 ; L = 8.650e+04 ; #DC iters = 0
#20 itrs. | Fval = -1.0317e+01 ; div. 7.04e-08 ; L = 8.650e+04 ; #DC iters = 0
#30 itrs. | Fval = -1.0591e+01 ; div. 1.56e-06 ; L = 1.709e+04 ; #DC iters = 0
#40 itrs. | Fval = -1.0895e+01 ; div. 2.59e-07 ; L = 3.844e+04 ; #DC iters = 0
#50 itrs. | Fval = -1.1325e+01 ; div. 1.90e-06 ; L = 1.709e+04 ; #DC iters = 0
#60 itrs. | Fval = -1.1684e+01 ; div. 2.45e-07 ; L = 3.844e+04 ; #DC iters = 0
#70 itrs. | Fval = -1.2172e+01 ; div. 1.14e-06 ; L = 1.709e+04 ; #DC iters = 0
#80 itrs. | Fval = -1.2674e+01 ; div. 6.40e-07 ; L = 2.563e+04 ; #DC iters = 0
#90 itrs. | Fval = -1.3094e+01 ; div. 5.71e-07 ; L = 2.563e+04 ; #DC iters = 0
#100 itrs. | Fval = -1.3620e+01 ; div. 2.76e-06 ; L = 1.139e+04 ; #DC iters = 0
#110 itrs. | Fval = -1.4550e+01 ; div. 3.24e-06 ; L = 1.139e+04 ; #DC iters = 0
#120 itrs. | Fval = -1.5354e+01 ; div. 1.29e

#50 itrs. | Fval = -1.1402e+01 ; div. 1.32e-06 ; L = 2.563e+04 ; #DC iters = 0
#60 itrs. | Fval = -1.1789e+01 ; div. 3.66e-07 ; L = 3.844e+04 ; #DC iters = 0
#70 itrs. | Fval = -1.2106e+01 ; div. 6.42e-07 ; L = 2.563e+04 ; #DC iters = 0
#80 itrs. | Fval = -1.2476e+01 ; div. 1.26e-06 ; L = 1.709e+04 ; #DC iters = 0
#90 itrs. | Fval = -1.2931e+01 ; div. 2.52e-06 ; L = 1.709e+04 ; #DC iters = 0
#100 itrs. | Fval = -1.3416e+01 ; div. 5.16e-06 ; L = 1.139e+04 ; #DC iters = 0
#110 itrs. | Fval = -1.3885e+01 ; div. 7.47e-06 ; L = 7.594e+03 ; #DC iters = 0
#120 itrs. | Fval = -1.4451e+01 ; div. 1.37e-05 ; L = 5.062e+03 ; #DC iters = 0
#130 itrs. | Fval = -1.5083e+01 ; div. 1.24e-06 ; L = 1.709e+04 ; #DC iters = 0
#140 itrs. | Fval = -1.6098e+01 ; div. 7.89e-06 ; L = 7.594e+03 ; #DC iters = 0
#150 itrs. | Fval = -1.6932e+01 ; div. 7.38e-07 ; L = 2.563e+04 ; #DC iters = 0
#160 itrs. | Fval = -1.7600e+01 ; div. 1.79e-06 ; L = 2.563e+04 ; #DC iters = 0
#170 itrs. | Fval = -1.8315e+01 ; div. 4.53e-

#40 itrs. | Fval = -6.7497e+01 ; div. 9.63e-02 ; L = 1.317e+02 ; #DC iters = 10 ; B_size = 2
#41 itrs. | Fval = -9.7981e+01 ; div. 3.54e-01 ; L = 8.779e+01 ; #DC iters = 11 ; B_size = 2
#42 itrs. | Fval = -2.2584e+02 ; div. 1.81e+00 ; L = 5.853e+01 ; #DC iters = 12 ; B_size = 3
#43 itrs. | Fval = -4.8733e+02 ; div. 4.88e+00 ; L = 3.902e+01 ; #DC iters = 13 ; B_size = 2
#44 itrs. | Fval = -1.5443e+03 ; div. 2.10e+01 ; L = 2.601e+01 ; #DC iters = 14 ; B_size = 2
#45 itrs. | Fval = -1.5443e+03 ; div. 0.00e+00 ; L = 1.734e+01 ; #DC iters = 14 ; B_size = 3
 
HALT #45 iters. | Fval = -1544.297715571919 ; div. 0.0 ; #DC iters = 14
 
go SDCe-LS
#10 itrs. | Fval = -9.3693e+00 ; div. 1.96e-06 ; L = 5.767e+04 ; #DC iters = 0
#20 itrs. | Fval = -1.0072e+01 ; div. 7.14e-06 ; L = 2.563e+04 ; #DC iters = 0
#30 itrs. | Fval = -1.1206e+01 ; div. 1.22e-05 ; L = 3.844e+04 ; #DC iters = 0
#40 itrs. | Fval = -1.3198e+01 ; div. 5.03e-05 ; L = 1.709e+04 ; #DC iters = 0
#50 itrs. | Fval = -1.5921e+01 ; div. 8

#10 itrs. | Fval = -1.4383e+01 ; div. 6.26e-05 ; L = 7.594e+03 ; #DC iters = 0 ; B_size = 3
#11 itrs. | Fval = -1.4653e+01 ; div. 5.60e-05 ; L = 1.139e+04 ; #DC iters = 0 ; B_size = 4
#12 itrs. | Fval = -1.4916e+01 ; div. 4.71e-05 ; L = 1.709e+04 ; #DC iters = 0 ; B_size = 4
#13 itrs. | Fval = -1.5154e+01 ; div. 4.22e-05 ; L = 1.709e+04 ; #DC iters = 0 ; B_size = 2
#14 itrs. | Fval = -1.5392e+01 ; div. 3.99e-05 ; L = 1.709e+04 ; #DC iters = 0 ; B_size = 2
#15 itrs. | Fval = -1.5643e+01 ; div. 4.62e-05 ; L = 1.139e+04 ; #DC iters = 0 ; B_size = 2
#16 itrs. | Fval = -1.5928e+01 ; div. 6.64e-05 ; L = 7.594e+03 ; #DC iters = 0 ; B_size = 2
#17 itrs. | Fval = -1.6276e+01 ; div. 1.06e-04 ; L = 5.062e+03 ; #DC iters = 0 ; B_size = 3
#18 itrs. | Fval = -1.6744e+01 ; div. 2.12e-04 ; L = 3.375e+03 ; #DC iters = 0 ; B_size = 2
#19 itrs. | Fval = -1.7358e+01 ; div. 3.30e-04 ; L = 3.375e+03 ; #DC iters = 0 ; B_size = 4
#20 itrs. | Fval = -1.8129e+01 ; div. 4.58e-04 ; L = 3.375e+03 ; #DC iters = 0 ;

...

#16 itrs. | Fval = -6.8670e+01 ; div. 1.47e-01 ; L = 1.317e+02 ; #DC iters = 8 ; B_size = 2
#17 itrs. | Fval = -1.0153e+02 ; div. 3.09e-01 ; L = 8.779e+01 ; #DC iters = 9 ; B_size = 2
#18 itrs. | Fval = -2.2227e+02 ; div. 1.80e+00 ; L = 5.853e+01 ; #DC iters = 10 ; B_size = 3
#19 itrs. | Fval = -8.4601e+02 ; div. 9.61e+00 ; L = 3.902e+01 ; #DC iters = 11 ; B_size = 2
#20 itrs. | Fval = -1.5677e+03 ; div. 1.80e+01 ; L = 2.601e+01 ; #DC iters = 12 ; B_size = 2
#21 itrs. | Fval = -1.5677e+03 ; div. 0.00e+00 ; L = 1.734e+01 ; #DC iters = 12 ; B_size = 3
 
HALT #21 iters. | Fval = -1567.7240307734448 ; div. 0.0 ; #DC iters = 12
 
go SDCe-LS
#10 itrs. | Fval = -1.5115e+01 ; div. 7.90e-06 ; L = 2.563e+04 ; #DC iters = 0
#20 itrs. | Fval = -1.6347e+01 ; div. 1.24e-05 ; L = 2.563e+04 ; #DC iters = 0
#30 itrs. | Fval = -1.9258e+01 ; div. 1.08e-04 ; L = 7.594e+03 ; #DC iters = 0
#40 itrs. | Fval = -2.4105e+01 ; div. 6.94e-04 ; L = 3.375e+03 ; #DC iters = 3
#50 itrs. | Fval = -6.6154e+01 ; div. 9.

#32 itrs. | Fval = -4.0052e+01 ; div. 2.70e-02 ; L = 2.963e+02 ; #DC iters = 11 ; B_size = 2
#33 itrs. | Fval = -5.3477e+01 ; div. 7.72e-02 ; L = 1.975e+02 ; #DC iters = 12 ; B_size = 2
#34 itrs. | Fval = -7.7924e+01 ; div. 1.68e-01 ; L = 1.317e+02 ; #DC iters = 13 ; B_size = 2
#35 itrs. | Fval = -1.3183e+02 ; div. 5.29e-01 ; L = 8.779e+01 ; #DC iters = 14 ; B_size = 2
#36 itrs. | Fval = -2.8084e+02 ; div. 2.06e+00 ; L = 5.853e+01 ; #DC iters = 15 ; B_size = 2
#37 itrs. | Fval = -7.2361e+02 ; div. 7.84e+00 ; L = 3.902e+01 ; #DC iters = 16 ; B_size = 2
#38 itrs. | Fval = -1.4605e+03 ; div. 1.71e+01 ; L = 2.601e+01 ; #DC iters = 17 ; B_size = 3
#39 itrs. | Fval = -1.4605e+03 ; div. 0.00e+00 ; L = 1.734e+01 ; #DC iters = 17 ; B_size = 4
 
HALT #39 iters. | Fval = -1460.4979074309333 ; div. 0.0 ; #DC iters = 17
 
go SDCe-LS
#10 itrs. | Fval = -1.3964e+01 ; div. 2.92e-06 ; L = 2.563e+04 ; #DC iters = 0
#20 itrs. | Fval = -1.4920e+01 ; div. 1.23e-05 ; L = 1.709e+04 ; #DC iters = 0
#30 itrs. 

#24 itrs. | Fval = -1.9942e+01 ; div. 1.18e-03 ; L = 1.500e+03 ; #DC iters = 4 ; B_size = 3
#25 itrs. | Fval = -2.1413e+01 ; div. 2.31e-03 ; L = 1.000e+03 ; #DC iters = 4 ; B_size = 2
#26 itrs. | Fval = -2.4023e+01 ; div. 5.87e-03 ; L = 6.667e+02 ; #DC iters = 5 ; B_size = 2
#27 itrs. | Fval = -2.7831e+01 ; div. 1.44e-02 ; L = 4.444e+02 ; #DC iters = 6 ; B_size = 2
#28 itrs. | Fval = -3.3950e+01 ; div. 2.31e-02 ; L = 2.963e+02 ; #DC iters = 6 ; B_size = 3
#29 itrs. | Fval = -4.3504e+01 ; div. 5.23e-02 ; L = 1.975e+02 ; #DC iters = 7 ; B_size = 2
#30 itrs. | Fval = -6.2007e+01 ; div. 1.48e-01 ; L = 1.317e+02 ; #DC iters = 8 ; B_size = 2
#31 itrs. | Fval = -1.1336e+02 ; div. 5.03e-01 ; L = 8.779e+01 ; #DC iters = 9 ; B_size = 2
#32 itrs. | Fval = -2.8460e+02 ; div. 2.52e+00 ; L = 5.853e+01 ; #DC iters = 10 ; B_size = 2
#33 itrs. | Fval = -9.1956e+02 ; div. 1.19e+01 ; L = 3.902e+01 ; #DC iters = 11 ; B_size = 2
#34 itrs. | Fval = -1.5492e+03 ; div. 8.31e+00 ; L = 2.601e+01 ; #DC iters = 1

#260 itrs. | Fval = -2.1393e+01 ; div. 9.43e-06 ; L = 7.594e+03 ; #DC iters = 0
#270 itrs. | Fval = -2.2518e+01 ; div. 1.50e-06 ; L = 1.709e+04 ; #DC iters = 0
#280 itrs. | Fval = -2.3687e+01 ; div. 7.52e-06 ; L = 7.594e+03 ; #DC iters = 0
#290 itrs. | Fval = -2.6131e+01 ; div. 1.62e-06 ; L = 2.563e+04 ; #DC iters = 0
#300 itrs. | Fval = -2.7503e+01 ; div. 1.92e-05 ; L = 5.062e+03 ; #DC iters = 0
#310 itrs. | Fval = -2.9577e+01 ; div. 1.52e-05 ; L = 7.594e+03 ; #DC iters = 0
#320 itrs. | Fval = -3.1168e+01 ; div. 5.18e-05 ; L = 3.375e+03 ; #DC iters = 0
#330 itrs. | Fval = -3.5045e+01 ; div. 3.12e-04 ; L = 1.500e+03 ; #DC iters = 0
#340 itrs. | Fval = -3.3158e+02 ; div. 1.80e+00 ; L = 2.601e+01 ; #DC iters = 0
 
HALT #344 iters. | Fval = -1549.2264673153234 ; div. 0.0 ; #DC iters = 0
 
repID: 4
 
 
go ASDCe
#1 itrs. | Fval = -1.3494e+01 ; div. 2.88e-04 ; L = 1.500e+03 ; #DC iters = 0 ; B_size = 3
#2 itrs. | Fval = -1.4094e+01 ; div. 1.04e-04 ; L = 2.250e+03 ; #DC iters = 0 ; B_size = 5

#170 itrs. | Fval = -2.7106e+01 ; div. 4.09e-05 ; L = 3.375e+03 ; #DC iters = 0
#180 itrs. | Fval = -2.8722e+01 ; div. 4.20e-05 ; L = 3.375e+03 ; #DC iters = 0
#190 itrs. | Fval = -3.0518e+01 ; div. 7.92e-06 ; L = 1.139e+04 ; #DC iters = 0
#200 itrs. | Fval = -3.9503e+01 ; div. 8.75e-03 ; L = 2.963e+02 ; #DC iters = 0
#210 itrs. | Fval = -1.4752e+03 ; div. 0.00e+00 ; L = 5.138e+00 ; #DC iters = 0
 
HALT #210 iters. | Fval = -1475.2429005943413 ; div. 0.0 ; #DC iters = 0
 
 


In [ ]:
import seaborn as sns
sns.set_theme()
sns.relplot(
    data=subdf, x="times", y="vals",
    hue="method", style="dc_updates",
    kind="scatter",col='extrapolation'
)

In [ ]:
for f in list(subdf['extrapolation'].unique()):
    subgroup = subdf[(subdf['extrapolation']==f)]
    sub = ['extrapolation: '+str(f)]
    print('subgroup: ('+str(sub)+')')
    print('avg. time: '+str(subgroup['times'].mean()))
    print('median time: '+str(subgroup['times'].median()))
    print('avg. val: '+str(subgroup['vals'].mean()))
    print('median val: '+str(subgroup['vals'].median()))
    print('avg. #iters: '+str(subgroup['iters'].mean()))
    print('median #iters: '+str(subgroup['iters'].median()))
    print(' ')

In [ ]:
for f in list(subdf['dc_updates'].unique()):
    subgroup = subdf[(subdf['dc_updates']==f)]
    sub = ['DC models: '+str(f)]
    print('subgroup: ('+str(sub)+')')
    print('avg. time: '+str(subgroup['times'].mean()))
    print('median time: '+str(subgroup['times'].median()))
    print('avg. val: '+str(subgroup['vals'].mean()))
    print('median val: '+str(subgroup['vals'].median()))
    print('avg. #iters: '+str(subgroup['iters'].mean()))
    print('median #iters: '+str(subgroup['iters'].median()))
    print(' ')

In [ ]:
for f in list(subdf['method'].unique()):
    subgroup = subdf[(subdf['method']==f)]
    sub = [f]
    print('subgroup: ('+str(sub)+')')
    print('avg. time: '+str(subgroup['times'].mean()))
    print('median time: '+str(subgroup['times'].median()))
    print('avg. val: '+str(subgroup['vals'].mean()))
    print('median val: '+str(subgroup['vals'].median()))
    print('avg. #iters: '+str(subgroup['iters'].mean()))
    print('median #iters: '+str(subgroup['iters'].median()))
    print(' ')